<a href="https://colab.research.google.com/github/Ayush-Sharma410/NLP/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PolyFuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from polyfuzz import PolyFuzz
from polyfuzz.models import RapidFuzz
import polyfuzz as pf
import numpy as np
import re
Netmeds=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Netmeds.csv"))
Pharmeasy=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Pharmeasy.csv"))
Apollo=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Apollopharmacy.csv"))
Mr=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Mrmeds.csv"))
Kauvery=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Kauverymeds.csv"))
tata1mg=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/tata1mg.csv"))
medkart=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/Medkart-Final.csv"))


Tata1mg_pharmeasy=pd.concat([Netmeds,Pharmeasy,Apollo,Mr,Kauvery,tata1mg,medkart],axis=0)


<ipython-input-8-ad18a9cb61ae>:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tata1mg=pd.DataFrame(pd.read_csv("/content/drive/MyDrive/new data/tata1mg.csv"))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Tata1mg_pharmeasy.columns

Index(['Index', 'id', 'Medicine_Link', 'Manufacturer', 'Real_Price', 'Salts',
       'Discounted_Price', 'Quantity', 'Prescription_Required',
       'Medicine_Name', 'Packaging', 'Quantity_', 'Form', 'Salt_Len',
       'All_Ingredients', 'Source', 'Availability', 'Salt1', 'Dosage1',
       'Salt2', 'Dosage2', 'Salt3', 'Dosage3', 'Salt4', 'Dosage4'],
      dtype='object')

In [ ]:
Tata1mg_pharmeasy.head()

,Index,id,Medicine_Link,Manufacturer,Real_Price,Salts,Discounted_Price,Quantity,Prescription_Required,Medicine_Name,...,Source,Availability,Salt1,Dosage1,Salt2,Dosage2,Salt3,Dosage3,Salt4,Dosage4
0,1,netmeds-1,https://www.netmeds.com/prescriptions/levikind...,Ikon Remedies,62.38,Levetiracetam,₹51.15,10 Tablet(s) in a Strip,1,levikind 250mg tablet 10s,...,netmeds,1,Levetiracetam,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,netmeds-2,https://www.netmeds.com/prescriptions/aquaoat-...,Emcure Pharmaceuticals,434.00,Skin Care + Liquid Paraffin + Glycerine + Shea...,₹355.88,NaN,0,aquaoat moisturizing cream 100gm,...,netmeds,1,Skin Care,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,netmeds-3,https://www.netmeds.com/prescriptions/pandural...,Pfizer,8.71,Benzathine Penicillin 6Mg,₹7.14,NaN,1,pandural la 6mg injection 1ml,...,netmeds,1,Benzathine Penicillin,6 mg,NaN,NaN,NaN,NaN,NaN,NaN
3,4,netmeds-4,https://www.netmeds.com/prescriptions/snatos-1...,Anax Life Sciences,110.00,Sertraline Hydrochloride 100 Mg,₹90.20,10 Tablet(s) in a Strip,1,snatos 100mg tablet 10s,...,netmeds,1,Sertraline Hydrochloride,100 mg,NaN,NaN,NaN,NaN,NaN,NaN
4,5,netmeds-5,https://www.netmeds.com/prescriptions/endolip-...,Lia Life Sciences,136.00,Atorvastatin Calcium 10 Mg + Fenofibrate 160 Mg,₹111.52,10 Tablet(s) in a Strip,1,endolip f tablet 10s,...,netmeds,1,Atorvastatin Calcium,10 mg,Fenofibrate,160 mg,NaN,NaN,NaN,NaN


In [ ]:
Tata1mg_pharmeasy['Medicine_Name'] = Tata1mg_pharmeasy['Medicine_Name'].str.lower()
Tata1mg_pharmeasy = Tata1mg_pharmeasy.sort_values(by="Medicine_Name")

In [ ]:
indexes_by_alphabet = {}

current_alphabet = None  # To keep track of the current alphabet
current_index = 0
for index, medicine_name in enumerate(Tata1mg_pharmeasy['Medicine_Name']):
    # Check if medicine_name is a string
    if isinstance(medicine_name, str):
        medicine_name=medicine_name.lower()
        first_letter = medicine_name[0]

        if first_letter != current_alphabet:
            # We've encountered a new alphabet, store its starting index
            indexes_by_alphabet[first_letter] = index
            current_alphabet = first_letter

# Now, 'indexes_by_alphabet' contains the starting index for each alphabet
print(indexes_by_alphabet)


{'(': 0, '0': 1, '1': 3, '2': 54, '3': 102, '4': 187, '5': 242, '6': 271, '7': 285, '8': 291, '9': 302, ':': 316, '@': 320, 'a': 325, 'b': 92484, 'c': 110108, 'd': 233314, 'e': 258004, 'f': 277864, 'g': 295890, 'h': 306163, 'i': 311297, 'j': 322675, 'k': 325218, 'l': 332799, 'm': 354509, 'n': 371614, 'o': 393207, 'p': 419313, 'q': 453772, 'r': 456771, 's': 492472, 't': 530033, 'u': 571357, 'v': 579186, 'w': 596989, 'x': 601504, 'y': 605252, 'z': 606154}


In [ ]:
def clean_dosage(x):
    if x==np.nan or type(x)==float:
        return 0
    else:
        x = x.lower()
        x = re.sub(r"[a-zA-Z]+","",x)
        x = x.replace(" ","")
        x = x.split("%")[0]
        x = x.split("/")[0]
        x = x.replace("(","")
        x = x.replace(")","")
        if x=="":
            return 0
        else:
          try:
            x = x.rsplit('.', 1)[0]
            x=float(x)
            return x
          except:
            return x

In [ ]:
def get_dosage(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    x4 = x[3]

    ds1 = clean_dosage(x1)
    ds2 = clean_dosage(x2)
    ds3 = clean_dosage(x3)
    ds4 = clean_dosage(x4)
    ds1 = str(ds1)
    ds2 = str(ds2)
    ds3 = str(ds3)
    ds4 = str(ds4)
    max_ds = max(ds1,ds2,ds3,ds4)

    if max_ds==ds1:
        return str(ds1)
    elif max_ds==ds2:
        return str(ds2)
    elif max_ds==ds3:
        return str(ds3)
    elif max_ds==ds4:
        return str(ds4)
    else:
        return 0

In [ ]:
start_alphabet = 'a'
end_alphabet = 'y'
start_integer = ord(start_alphabet) - ord('a')
end_integer = ord(end_alphabet) - ord('a')+1
alphabets_as_integers = list(range(start_integer, end_integer + 1))

def process_name(name):
    words = name.split()
    if len(words) > 0 and len(words[0]) < 2:
        return ' '.join(words[:])
    return words[0]
def sort_cluster_by_form(cluster_data):
    return cluster_data.sort_values(by='Form')

mapping_dict = {
    0: 'a', 1: 'b', 2: 'c', -1: 'a', 3: 'd', 4: 'e', 5: 'f',
    6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm',
    13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u'
}
# Loop over each integer and save the corresponding file
for alphabet_integer in alphabets_as_integers:
    current_alphabet = chr(ord('a') + alphabet_integer)
    Ddata = Tata1mg_pharmeasy.iloc[indexes_by_alphabet[current_alphabet]:indexes_by_alphabet[chr(ord(current_alphabet) +1)], :]

    Ddata['merged'] = Ddata['Medicine_Name'].apply(process_name)
    merge_list_lower = [str(item).lower() for item in Ddata['merged']]
    merged_list = pd.DataFrame(merge_list_lower)
    matcher = RapidFuzz(n_jobs=1, score_cutoff=0.90)
    model = PolyFuzz(matcher)
    model.match(merge_list_lower)
    model.group()
    df_initial=pd.DataFrame(model.get_matches())
    df_groups=model.get_matches().iloc[:,-1].values
    df_initial.rename(columns={'From':'from'},inplace=True)

    none_indices = df_initial[df_initial["Group"].isnull()].index


    # Assign cluster names to items with "None" group
    for idx in none_indices:

        df_initial.iloc[idx,3] = df_initial.iloc[idx,0]

    # Print unique values in the "Group" column
    unique_groups = len(df_initial["Group"].unique())

    df2=set(df_initial['Group'].values)
    string_to_value = {string: value for value, string in enumerate(list(df2))}

    new = pd.DataFrame.from_dict(string_to_value,orient ='index')
    df_initial['cluster']=df_initial.Group.map(string_to_value)
    Ddata['cluster']=Ddata.merged.map(string_to_value)
    sorted_df = Ddata.sort_values(by='cluster', ascending=True)
    sorted_df.reset_index(inplace=True, drop=True)
    try:
      sorted_df[sorted_df['cluster'].isna()].index[0]

      sorteddf2 = sorted_df.iloc[sorted_df[sorted_df['cluster'].isna()].index[0]: ,:]
      df3=set(sorteddf2 ['merged'].values)
      string_to_value = {string: value for value, string in enumerate(list(df3), start=int(float(max(sorted_df['cluster'])) + 1))}

      new1 = pd.DataFrame.from_dict(string_to_value,orient ='index')
      sorted_df['cluster1']=sorted_df.merged.map(string_to_value)
      sorted_df['cluster_id'] = sorted_df['cluster'].fillna(sorted_df['cluster1']).astype(int)
      sorted_df.sort_values(by='cluster_id',inplace =True)
      sorted_df.drop(['cluster', 'cluster1'], axis=1, inplace=True)

      refined_clusters = []

      for cluster_id in set(sorted_df['cluster_id']):
          # Extract data points belonging to the current cluster
          cluster_data = sorted_df[sorted_df['cluster_id'] == cluster_id]

          # Sort the cluster data by the 'Form' column
          sorted_cluster_data = sort_cluster_by_form(cluster_data)

          # Create a new cluster ID based on the 'Form' column
          sorted_cluster_data['subcluster_id'] = sorted_cluster_data['Form'].factorize()[0]

          # Append the sorted cluster data to the final result
          refined_clusters.append(sorted_cluster_data)

      # Combine the refined clusters into a single DataFrame
      refined_data = pd.concat(refined_clusters)
      refined_data['subcluster_id']=refined_data['subcluster_id'].map(mapping_dict)
      refined_data['new_clusters'] = refined_data['cluster_id'].astype(str) + refined_data['subcluster_id'].astype(str)
      refined_data.drop(columns=['subcluster_id'],inplace=True)
      refined_data['Dosage'] = refined_data[['Dosage1','Dosage2','Dosage3','Dosage4']].apply(lambda x: get_dosage(x),axis=1)
      refined_data['generic_cluster_name'] = refined_data['merged']+" "+refined_data['Dosage']+" "+refined_data['Form']
      refined_data = refined_data.rename(columns={'merged': 'Name Prefixes'})
      refined_data.to_csv(f"/content/drive/MyDrive/Clustered-data-Final/clustered_data({current_alphabet.upper()}).csv",index=False)
    except:
            refined_clusters = []

            for cluster_id in set(sorted_df['cluster_id']):
                # Extract data points belonging to the current cluster
                cluster_data = sorted_df[sorted_df['cluster_id'] == cluster_id]

                # Sort the cluster data by the 'Form' column
                sorted_cluster_data = sort_cluster_by_form(cluster_data)

                # Create a new cluster ID based on the 'Form' column
                sorted_cluster_data['subcluster_id'] = sorted_cluster_data['Form'].factorize()[0]

                # Append the sorted cluster data to the final result
                refined_clusters.append(sorted_cluster_data)

            # Combine the refined clusters into a single DataFrame
            refined_data = pd.concat(refined_clusters)
            refined_data['subcluster_id']=refined_data['subcluster_id'].map(mapping_dict)
            refined_data['new_clusters'] = refined_data['cluster_id'].astype(str) + refined_data['subcluster_id'].astype(str)
            refined_data.drop(columns=['subcluster_id'],inplace=True)
            refined_data['Dosage'] = refined_data[['Dosage1','Dosage2','Dosage3','Dosage4']].apply(lambda x: get_dosage(x),axis=1)
            refined_data['generic_cluster_name'] = refined_data['merged']+" "+refined_data['Dosage']+" "+refined_data['Form']
            refined_data = refined_data.rename(columns={'merged': 'Name Prefixes'})
            refined_data.to_csv(f"/content/drive/MyDrive/Clustered-data-Final/clustered_data({current_alphabet.upper()}).csv",index=False)



In [ ]:
def process_name(name):
    name = str(name)
    words = name.split()
    if len(words) > 0 and len(words[0]) < 2:
        return ' '.join(words[:])
    return words[0]
def sort_cluster_by_form(cluster_data):
    return cluster_data.sort_values(by='Form')

mapping_dict = {
    0: 'a', 1: 'b', 2: 'c', -1: 'a', 3: 'd', 4: 'e', 5: 'f',
    6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm',
    13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u'
}

Ddata = Tata1mg_pharmeasy.iloc[606154:, :]

Ddata['merged'] = Ddata['Medicine_Name'].apply(process_name)
merge_list_lower = [str(item).lower() for item in Ddata['merged']]
merged_list = pd.DataFrame(merge_list_lower)


matcher = RapidFuzz(n_jobs=1, score_cutoff=0.90)
model = PolyFuzz(matcher)
model.match(merge_list_lower)
model.group()
df_initial=pd.DataFrame(model.get_matches())
df_groups=model.get_matches().iloc[:,-1].values
df_initial.rename(columns={'From':'from'},inplace=True)

none_indices = df_initial[df_initial["Group"].isnull()].index


# Assign cluster names to items with "None" group
for idx in none_indices:

  df_initial.iloc[idx,3] = df_initial.iloc[idx,0]

# Print unique values in the "Group" column
unique_groups = len(df_initial["Group"].unique())

df2=set(df_initial['Group'].values)
string_to_value = {string: value for value, string in enumerate(list(df2))}

new = pd.DataFrame.from_dict(string_to_value,orient ='index')
df_initial['cluster']=df_initial.Group.map(string_to_value)
Ddata['cluster']=Ddata.merged.map(string_to_value)
sorted_df = Ddata.sort_values(by='cluster', ascending=True)
sorted_df.reset_index(inplace=True, drop=True)
try:
      sorted_df[sorted_df['cluster'].isna()].index[0]

      sorteddf2 = sorted_df.iloc[sorted_df[sorted_df['cluster'].isna()].index[0]: ,:]
      df3=set(sorteddf2 ['merged'].values)
      string_to_value = {string: value for value, string in enumerate(list(df3), start=int(float(max(sorted_df['cluster'])) + 1))}

      new1 = pd.DataFrame.from_dict(string_to_value,orient ='index')
      sorted_df['cluster1']=sorted_df.merged.map(string_to_value)
      sorted_df['cluster_id'] = sorted_df['cluster'].fillna(sorted_df['cluster1']).astype(int)
      sorted_df.sort_values(by='cluster_id',inplace =True)
      sorted_df.drop(['cluster', 'cluster1'], axis=1, inplace=True)

      refined_clusters = []

      for cluster_id in set(sorted_df['cluster_id']):
          # Extract data points belonging to the current cluster
          cluster_data = sorted_df[sorted_df['cluster_id'] == cluster_id]

          # Sort the cluster data by the 'Form' column
          sorted_cluster_data = sort_cluster_by_form(cluster_data)

          # Create a new cluster ID based on the 'Form' column
          sorted_cluster_data['subcluster_id'] = sorted_cluster_data['Form'].factorize()[0]

          # Append the sorted cluster data to the final result
          refined_clusters.append(sorted_cluster_data)

      # Combine the refined clusters into a single DataFrame
      refined_data = pd.concat(refined_clusters)
      refined_data['subcluster_id']=refined_data['subcluster_id'].map(mapping_dict)
      refined_data['new_clusters'] = refined_data['cluster_id'].astype(str) + refined_data['subcluster_id'].astype(str)
      refined_data.drop(columns=['subcluster_id'],inplace=True)
      refined_data['Dosage'] = refined_data[['Dosage1','Dosage2','Dosage3','Dosage4']].apply(lambda x: get_dosage(x),axis=1)
      refined_data['generic_cluster_name'] = refined_data['merged']+" "+refined_data['Dosage']+" "+refined_data['Form']
      refined_data = refined_data.rename(columns={'merged': 'Name Prefixes'})
      refined_data.to_csv(f"/content/drive/MyDrive/Clustered-data-Final/clustered_data(Z).csv",index=False)
except:
      refined_clusters = []

      for cluster_id in set(sorted_df['cluster_id']):
          # Extract data points belonging to the current cluster
          cluster_data = sorted_df[sorted_df['cluster_id'] == cluster_id]

          # Sort the cluster data by the 'Form' column
          sorted_cluster_data = sort_cluster_by_form(cluster_data)

          # Create a new cluster ID based on the 'Form' column
          sorted_cluster_data['subcluster_id'] = sorted_cluster_data['Form'].factorize()[0]

          # Append the sorted cluster data to the final result
          refined_clusters.append(sorted_cluster_data)

      # Combine the refined clusters into a single DataFrame
      refined_data = pd.concat(refined_clusters)
      refined_data['subcluster_id']=refined_data['subcluster_id'].map(mapping_dict)
      refined_data['new_clusters'] = refined_data['cluster_id'].astype(str) + refined_data['subcluster_id'].astype(str)
      refined_data.drop(columns=['subcluster_id'],inplace=True)
      refined_data['Dosage'] = refined_data[['Dosage1','Dosage2','Dosage3','Dosage4']].apply(lambda x: get_dosage(x),axis=1)
      refined_data['generic_cluster_name'] = refined_data['merged']+" "+refined_data['Dosage']+" "+refined_data['Form']
      refined_data = refined_data.rename(columns={'merged': 'Name Prefixes'})
      refined_data.to_csv(f"/content/drive/MyDrive/Clustered-data-Final/clustered_data(Z).csv",index=False)

<ipython-input-15-22254e6f2d1e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ddata['merged'] = Ddata['Medicine_Name'].apply(process_name)
<ipython-input-15-22254e6f2d1e>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ddata['cluster']=Ddata.merged.map(string_to_value)


In [ ]:
import pandas as pd

# Create an empty list to store DataFrames
dfs = []

# Iterate through the alphabets A to Z
for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    # Assuming you have the file path pattern
    file_path = f'/content/drive/MyDrive/Clustered-data-Final/clustered_data({letter}).csv'

    # Read the CSV file and append it to the list of DataFrames
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate the DataFrames row-wise
combined_df = pd.concat(dfs, ignore_index=True)

# Now, 'combined_df' contains all the rows from the individual DataFrames


<ipython-input-16-af8e9978c401>:12: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-16-af8e9978c401>:12: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-16-af8e9978c401>:12: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
combined_df.rename(columns={'new_clusters': 'sub_cluster_id'}, inplace=True)


In [ ]:
combined_df.to_csv('/content/drive/MyDrive/A-Z Clustered/a-z clustered.csv',index=False)

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/A-Z Clustered/a-z clustered.csv")

<ipython-input-58-5e754a31d894>:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/drive/MyDrive/A-Z Clustered/a-z clustered.csv")


In [ ]:
data.columns

Index(['Index', 'id', 'Medicine_Link', 'Manufacturer', 'Real_Price', 'Salts',
       'Discounted_Price', 'Quantity', 'Prescription_Required',
       'Medicine_Name', 'Packaging', 'Quantity_', 'Form', 'Salt_Len',
       'All_Ingredients', 'Source', 'Availability', 'Salt1', 'Dosage1',
       'Salt2', 'Dosage2', 'Salt3', 'Dosage3', 'Salt4', 'Dosage4',
       'Name Prefixes', 'cluster_id', 'sub_cluster_id', 'Dosage',
       'generic_cluster_name'],
      dtype='object')

In [ ]:
data.iloc[:100,]

,Index,id,Medicine_Link,Manufacturer,Real_Price,Salts,Discounted_Price,Quantity,Prescription_Required,Medicine_Name,...,Dosage2,Salt3,Dosage3,Salt4,Dosage4,Name Prefixes,cluster_id,sub_cluster_id,Dosage,generic_cluster_name
0,67851,tata1mg-67851,https://www.1mg.com/drugs/amitop-plus-h-tablet...,Tas Med India,27.5,Amitriptyline (12.5mg) + Chlordiazepoxide (5mg),NaN,NaN,1,apsolam 0.5mg tablet,...,5mg,NaN,NaN,NaN,NaN,apsolam,0,0a,5.0,apsolam 5.0 tablet
1,7851,tata1mg-7851,https://www.1mg.com/drugs/apsolam-0.5mg-tablet...,Aps Biotech,22.0,Alprazolam (0.5mg),NaN,NaN,1,apsolam 0.5mg tablet,...,NaN,NaN,NaN,NaN,NaN,apsolam,0,0a,5.0,apsolam 5.0 tablet
2,27851,tata1mg-27851,https://www.1mg.com/drugs/danclear-gel-276991,Zydus Cadila,197.8,Ketoconazole (2% w/v) + Zinc pyrithione (1% w/v),176.0,90 gm in 1 tube,0,apsolam 0.5mg tablet,...,NaN,NaN,NaN,NaN,NaN,apsolam,0,0a,0.0,apsolam 0 tablet
3,107851,tata1mg-107851,https://www.1mg.com/drugs/lofequik-8-mg-500-mg...,Sun Pharmaceutical Industries,80.0,Lornoxicam (8mg) + Paracetamol (500mg),NaN,NaN,1,apsolam 0.5mg tablet,...,500mg,NaN,NaN,NaN,NaN,apsolam,0,0a,8.0,apsolam 8.0 tablet
4,47851,tata1mg-47851,https://www.1mg.com/drugs/ficsi-200mg-tablet-7...,Modi Mundi Pharma,163.0,Cefixime (200mg),NaN,Strip 10 tablets in 1 strip,1,apsolam 0.5mg tablet,...,NaN,NaN,NaN,NaN,NaN,apsolam,0,0a,200.0,apsolam 200.0 tablet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,22148,pharmeasy-22148,https://pharmeasy.in/online-medicine-order/azi...,Azillian Healthcare,123.2,Ginseng(21.25 Mg) + Gingko Biloba(20.0 Mg) + V...,104.72,10 Capsule(s) in Strip,1,azivital 4g strip of 10 capsules,...,20.0 Mg,Vitamin B9 / Folic Acid / Folate,50.0 Mcg,Elemental Zinc,12.0 Mg,azivital,21,21a,50.0,azivital 50.0 capsule
96,6665,tata1mg-6665,https://www.1mg.com/drugs/azivital-500mg-table...,Nourish Healthcare,53.0,Azithromycin (500mg),49.0,3 tablets in 1 strip,1,azivital 500mg tablet,...,NaN,NaN,NaN,NaN,NaN,azivital,21,21b,500.0,azivital 500.0 tablet
97,66665,tata1mg-66665,https://www.1mg.com/drugs/aceclomax-sp-tablet-...,Growmax Medicare,75.0,Aceclofenac (100mg) + Paracetamol (325mg) + Se...,69.0,10 tablets in 1 strip,1,azivital 500mg tablet,...,325mg,Serratiopeptidase,10mg,NaN,NaN,azivital,21,21b,325.0,azivital 325.0 tablet
98,126665,tata1mg-126665,https://www.1mg.com/drugs/medoxi-cv-ds-dry-syr...,Medbionic Healthcare,117.0,Amoxycillin (400mg/5ml) + Clavulanic Acid (57m...,108.0,30 ml in 1 bottle,1,azivital 500mg tablet,...,57mg,NaN,NaN,NaN,NaN,azivital,21,21b,57.0,azivital 57.0 tablet


In [ ]:
data['new_subcluster_id'] = data['Dosage'].factorize()[0]

In [ ]:
mapping_dict = {
    np.nan:'a',0: 'a', 1: 'b', 2: 'c', -1: 'a', 3: 'd', 4: 'e', 5: 'f',
    6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm',
    13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u'
}

In [ ]:
data['new_subcluster_id']=data['new_subcluster_id'].map(mapping_dict)

In [ ]:
import numpy as np

mapping_dict[np.nan] = 'a'

In [ ]:
mapping_dict

{nan: 'a',
 0: 'a',
 1: 'b',
 2: 'c',
 -1: 'a',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u'}

In [ ]:
data['new_subcluster_id'] = data['sub_cluster_id'].astype(str) + data['new_subcluster_id'].astype(str)

In [ ]:
data['new_subcluster_id']

0              0aa
1              0aa
2              0ab
3              0ac
4              0ad
            ...   
627312    4918anan
627313      4919ad
627314    4920anan
627315      4921am
627316    4922anan
Name: new_subcluster_id, Length: 627317, dtype: object

In [ ]:
data.iloc[:,25:]

,Name Prefixes,cluster_id,sub_cluster_id,Dosage,generic_cluster_name,new_subcluster_id
0,apsolam,0,0a,5.0,apsolam 5.0 tablet,0aa
1,apsolam,0,0a,5.0,apsolam 5.0 tablet,0aa
2,apsolam,0,0a,0.0,apsolam 0 tablet,0ab
3,apsolam,0,0a,8.0,apsolam 8.0 tablet,0ac
4,apsolam,0,0a,200.0,apsolam 200.0 tablet,0ad
...,...,...,...,...,...,...
627312,zensome,4918,4918a,40.0,zensome 40.0 capsule,4918anan
627313,z pod dt 200mg tablets,4919,4919a,200.0,z pod dt 200mg tablets 200.0 tablet,4919ad
627314,zelmox,4920,4920a,60.0,zelmox 60.0 capsule,4920anan
627315,zodal,4921,4921a,6.0,zodal 6.0 tablet,4921am


In [ ]:
data.to_csv("/content/drive/MyDrive/A-Z Clustered/new-clustered.csv",index=False)